# Run this Notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DeepRLCourse/Homework-3-Questions/blob/main/HW3_P3_MountainCarContinuous_REINFORCE.ipynb)  
[![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/DeepRLCourse/Homework-3-Questions/blob/main/HW3_P3_MountainCarContinuous_REINFORCE.ipynb)

# HW3: REINFORCE with continous action space in MountainCarContinous
> - Full Name: **Parsa Ghezelbash**
> - Student ID: **401110437**


This notebook implements the **REINFORCE policy gradient algorithm** to train an agent in the **MountainCarContinuous-v0** environment. The objective is to evaluate the performance and stability of REINFORCE in a continuous action space.

Follow the instructions in each section to complete the homework.

**Grading Breakdown:**

- Practical Implementation: 80 points
- Conceptual Understanding: 20 points

# Setup

All required packages are pre-installed if using Google Colab.

Import the following libraries.

In [37]:
# Required Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
import gym
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import base64
import imageio
import IPython
import logging
import warnings

# Disable warnings
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# DEVICE
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

Configure Matplotlib for Interactive and XKCD-Style Plots

In [38]:
# Set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display
plt.ion()
plt.xkcd(scale=1, length=100, randomness=2)
matplotlib.rcParams['figure.figsize'] = (12, 6)

Record and Embed Simulation Videos in Jupyter Notebook

<b>embed_mp4:</b> Converts an MP4 video into a base64-encoded HTML tag for display in Jupyter Notebook.
<br>
<b>record_simulation:</b> Runs a policy in the environment, records the simulation, and saves it as an MP4 video.

In [39]:
def embed_mp4(filename):
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())
    return IPython.display.HTML(tag)

def record_simulation(env, policy_net, filename, episodes=1, fps=30):
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
        for _ in range(episodes):
            state = env.reset()
            frame = env.render()  # Capture the first frame
            video.append_data(frame[0])

            done = False
            while not done:
                state_tensor = torch.FloatTensor(state).to(DEVICE)
                mean, _ = policy_net(state_tensor)
                action = mean.item()
                state, _, terminated, truncated, _ = env.step([action])

                frame = env.render()  # Capture the frame after taking the action
                video.append_data(frame[0])

                done = terminated or truncated
    return embed_mp4(filename)

# Explore the environment (10 points)

Initialize MountainCarContinuous Environment and Display State & Action Spaces

In [40]:
# Create the MountainCarContinuous environment
env = gym.make("MountainCarContinuous-v0")

# Print observation and action space
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)

Observation Space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Action Space: Box(-1.0, 1.0, (1,), float32)


**Question 1: (5 points)**

How are the observation and action spaces defined in the MountainCarContinuous environment?

Observation space contains position and velocity of the Car.</br>
Action space the force applied to the car and ranges in [-1, 1]</br>
$S=\{(x,v)∣x∈[−1.2, -0.07],v∈[0.06, 0.07]\}$</br>
$A=\{a∣a∈[−1.0,1.0]\}$</br>


Define a Random Policy for Action Selection  

In [41]:
class RandomPolicy(object):
    def __init__(self, action_space_n):
        self.action_space_n = action_space_n

    def __call__(self, state):
        action_values, nan = torch.rand(self.action_space_n), np.nan
        return action_values, nan

Visualize the random policy.

In [42]:
random_policy = RandomPolicy(env.action_space.shape[0])

record_simulation(gym.make("MountainCarContinuous-v0", render_mode='rgb_array', new_step_api=True), random_policy, "Video_MountainCarContinuous_random_policy")

**Question 2: (5 points)**

How could an agent reach the goal in the MountainCarContinuous environment while using the least amount of energy? Explain a scenario describing the agent's behavior during an episode with most optimal policy.

...</br>
The optimal policy for using least amount of energy is such that the Car pushes to left and then uses gravity to gain momentum and then use a littlee energy on when it reaches the rightmost point of the valley to reach the goal.</br>
...</br>


# Agent with REINFORCE in a continuous action space in MountainCarContinuous-v0 (80 points)

### Policy Network Definition (10 points)

In continuous action spaces, instead of choosing from a discrete set of actions, the policy network learns a Gaussian distribution
$\pi_\theta(a|s)$ over possible actions. The agent then samples an action from this distribution:

$$a \sim \mathcal{N}(\mu_\theta(s), \sigma_\theta(s))$$

where:

- $\mu_\theta(s)$ is the mean of the action distribution.
- $\sigma_\theta(s)$ is the standard deviation (exploration factor).

The network is trained using policy gradients, reinforcing actions that lead to better rewards.


In [47]:
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.mean_layer = nn.Linear(128, action_dim)
        self.log_std = nn.Parameter(torch.zeros(action_dim))

    def forward(self, x):
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        mean = F.tanh(self.mean_layer(x))
        std = torch.exp(self.log_std)
        return mean, std

### Computing Discounted Returns (15 points)

To implement a function that calculates the discounted return for each timestep in an episode.

$$[
G_t = \sum_{k=0}^{T-t} \gamma^k R_{t+k}
]$$

In [48]:
def compute_returns(rewards, gamma=0.99):
    returns = []
    G = 0
    for r in reversed(rewards):
        G = r + gamma * G
        returns.append(G)
    returns = list(reversed(returns))
    return returns

### Implementing the REINFORCE Algorithm in continuous action space (45 points)

To train the agent using the standard policy gradient method.
The REINFORCE algorithm updates policy parameters by using the log-probability of actions multiplied by the discounted return.

This algorithm optimizes a **stochastic policy** $( \pi_{\theta}(a_t \mid s_t) )$ by updating its parameters in the direction that increases expected rewards. The update rule is based on the **policy gradient theorem**:  

$$[
\theta \leftarrow \theta + \alpha \sum_{t=0}^{T} \nabla_{\theta} \log \pi_{\theta}(a_t \mid s_t) G_t
]$$

where:  

- $( \theta )$ are the policy parameters (weights of the neural network).  
- $( \alpha )$ is the learning rate.  
- $( G_t )$ is the **discounted return** from timestep $( t )$:  

- $( \nabla_{\theta} \log \pi_{\theta}(a_t \mid s_t) )$ is the gradient of the log-probability of the selected action, used to adjust the policy in the correct direction.

In [49]:
def reinforce(env, policy_net, optimizer, num_episodes=1000, gamma=0.99):
    rewards_per_episode = []
    for episode in range(num_episodes):
        state = env.reset()
        trajectory, rewards = [], []

        while True:
            state_tensor = torch.tensor(state, dtype=torch.float32).to(DEVICE)
            with torch.no_grad():
                mean, std = policy_net(state_tensor)
            action = Normal(mean, std).sample().cpu().numpy()

            new_state, reward, done, _ = env.step(action)

            trajectory.append((state, action, reward))
            rewards.append(reward)

            state = new_state

            if done:
                break

        returns = compute_returns(rewards, gamma)
        returns = torch.tensor(returns, dtype=torch.float32).to(DEVICE)
        # returns = (returns - returns.mean()) / (returns.std() + 1e-8)

        loss = 0
        for (state, action, _), G in zip(trajectory, returns):
            state_tensor = torch.tensor(state, dtype=torch.float32).to(DEVICE)
            mean, std = policy_net(state_tensor)
            action_tensor = torch.tensor(action, dtype=torch.float32).to(DEVICE)
            log_probs = Normal(mean, std).log_prob(action_tensor)
            loss -= log_probs * G

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        rewards_per_episode.append(sum(rewards))

        if (episode + 1) % 50 == 0:
            print(f"Episode {episode + 1}, Reward: {sum(rewards)}")

    return rewards_per_episode

### Training (10 points)

To define key hyperparameters that influence training efficiency and convergence.

Hyperparameters like learning rate, discount factor, and number of episodes significantly impact the training process. Proper tuning is necessary to ensure stable learning while avoiding slow convergence or premature convergence to suboptimal policies.

In [50]:
INPUT_DIM = env.observation_space.shape[0]
OUTPUT_DIM = env.action_space.shape[0]
LEARNING_RATE = 0.0005
DISCOUNT_FACTOR = 0.99
NUM_EPISODES = 1000

# Initialize Policy
policy = PolicyNetwork(INPUT_DIM, OUTPUT_DIM).to(DEVICE)

# Optimizer
optimizer= optim.Adam(policy.parameters(), lr=LEARNING_RATE)

# Training
print("Training REINFORCE Algorithm for Continuous Action Space:")
rewards = reinforce(env, policy, optimizer, num_episodes=NUM_EPISODES, gamma=DISCOUNT_FACTOR)

Training REINFORCE Algorithm for Continuous Action Space:


<ipython-input-49-d5d12a38277b>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action_tensor = torch.tensor(action, dtype=torch.float32).to(DEVICE)


Episode 50, Reward: -65.21551188233764
Episode 100, Reward: -68.06910274053608
Episode 150, Reward: -67.00623878978463
Episode 200, Reward: -50.57469921443606
Episode 250, Reward: -56.82168684630514
Episode 300, Reward: -54.55246699630591
Episode 350, Reward: -48.23109076075277
Episode 400, Reward: -45.61157296197647
Episode 450, Reward: -52.24069851148353
Episode 500, Reward: -44.12072836636093
Episode 550, Reward: -54.77980670982773
Episode 600, Reward: -45.06528887830009
Episode 650, Reward: -40.34155410888837
Episode 700, Reward: -38.753471499291535


KeyboardInterrupt: 

# Results (10 points)

In [ ]:
# Plotting Results
window_size = 50  # Moving average window size

# Exponential Moving Average calculation
def exponential_moving_average(data, window_size):
    alpha = 2 / (window_size + 1)  # Smoothing factor
    return np.convolve(data, [alpha * (1 - alpha) ** i for i in range(window_size)], mode='valid')

plt.plot(rewards, label="REINFORCE", alpha=0.5, color='tab:green')

# Plotting exponential moving averages
plt.plot(exponential_moving_average(rewards, window_size), label="Exponential Moving Avg", linestyle='--', color='tab:green')

plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.legend()
plt.title("REINFORCE: MountainCarContinuous")
plt.show()

In [ ]:
mean_reward, std_reward = np.mean(rewards), np.std(rewards)
print(f"mean_reward = {mean_reward:.2f} +/- {std_reward:.4f}")

**Question 3: (10 points)**

What strategies can be employed to reduce catastrophic forgetting in continuous action space environments like MountainCarContinuous?
</br>(Hint: experience replay or target networks)

.....</br>
.....</br>


# Simulation

In [ ]:
# Record the simulation using the optimal policy
record_simulation(gym.make("MountainCarContinuous-v0", render_mode='rgb_array', new_step_api=True), policy, "Video_MountainCarContinuous_REINFORCE")